In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [22]:
file_path = "../visualization/example_log.csv"

x_names = ['L%d%c' % (i, c) for i in range(21) for c in ['x', 'y', 'z']]
y_names = ['a', 's', 'd', 'f']

col_names = x_names + y_names

df = pd.read_csv(file_path,names=col_names)

In [23]:
import sys, os
sys.path.append(os.pardir)

from visualization.smoother import gamma_smoothing, average_smoothing

In [24]:
df = average_smoothing(df)

C:\Users\honey\anaconda3\envs\sib\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [25]:
scaler = MinMaxScaler()

for column in x_names:
	df_column = df[column]
	df_column = df_column.values.reshape(-1, 1)
	df_column = scaler.fit_transform(df_column)

	for row in range(len(df)):
		df[column].iloc[row] = df_column[row]


nda3\envs\sib\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\honey\anaconda3\envs\sib\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\honey\anaconda3\envs\sib\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [26]:
x = df.iloc[:,:63]
y = df.iloc[:,63:]

In [27]:
x_train, x_test = x.iloc[:500,:], x.iloc[500:,:]
y_train, y_test = y.iloc[:500,:], y.iloc[500:,:]

In [28]:
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [29]:
x_train_t = x_train.reshape(x_train.shape[0],63,1)
x_test_t = x_test.reshape(x_test.shape[0],63,1)

In [30]:
model = Sequential()

model.add(LSTM(20, input_shape=(63,1)))
model.add(Dense(64))
model.add(Dense(4))
model.compile(loss='mean_squared_error',optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1344      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 3,364
Trainable params: 3,364
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(x_train_t, y_train, epochs=20)

Epoch 1/20
16/16 [==============================] - 3s 12ms/step - loss: 0.0951
Epoch 2/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0958
Epoch 3/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0960
Epoch 4/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0937
Epoch 5/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0984
Epoch 6/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0967
Epoch 7/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0922
Epoch 8/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0952
Epoch 9/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0933
Epoch 10/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0974
Epoch 11/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0923
Epoch 12/20
16/16 [==============================] - 0s 12ms/step - loss: 0.0983
Epoch 13/20
16/16 [==================

In [32]:
model.evaluate(x_test_t, y_test)
OMG = model.predict(x_test_t)

9/9 [==============================] - 1s 4ms/step - loss: 0.1355


In [33]:
for i in range(len(x_test_t)):
    print(OMG[i].argmax(), y_test[i])

3 [1 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 1]
3 [0 0 0 1]
3 [0 0 0 1]
3 [0 0 0 1]
3 [0 0 0 1]
3 [0 0 0 1]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 1 0]
3 [0 0 1 0]
3 [0 0 1 0]
3 [0 0 1 0]
3 [0 0 1 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 1]
3 [0 0 0 1]
3 [0 0 0 1]
3 [0 0 0 1]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 1 0]
3 [0 0 1 0]
3 [0 0 1 0]
3 [0 0 1 0]
3 [0 0 1 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 1 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [0 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [1 0 0 0]
3 [0